In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.models import thresholdingRadarSum
from src.post_analysis import WeightChangeCalculator
from src.data import load_annotation
from src.utils import apply_double_median_filter, apply_median_filter
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(category=RuntimeWarning, action='ignore')

In [3]:
annotations = load_annotation.get_annotation()
defecate_ids = load_annotation.get_complete_ids('Defecation')

In [4]:
len(defecate_ids)

91

In [5]:
config = {
    'USER_ID': 0,
    'FILTER_WINDOW_SIZE': 11,
    'THRESHOLD': 0.18,
    'LEFT_EXTENSION_TIME': 0.6,
    'RIGHT_EXTENSION_TIME': 0.8
}

error_rates = []
true_intervals = []
pred_intervals = []
trues = []
preds = []

In [6]:
test = 0
for user_id in tqdm(defecate_ids[defecate_ids > 1000]):
#     test += 1
#     if test > 10:
#         break
    try:
        config['USER_ID'] = user_id
        model = thresholdingRadarSum.ThresholdingRadarSum(config)
        calculator = WeightChangeCalculator(user_id)

        a_intervals = [[e.start, e.stop]
                       for e in annotations[user_id] if e.event == 'Defecation']
        p_intervals = model.get_pred_intervals()

        true_weight_change = calculator.get_weight_change(
            start_stop_list=a_intervals,
            smooth_method=apply_double_median_filter,
            window_size=21
        )
        pred_weight_change = calculator.get_weight_change(
            start_stop_list=p_intervals,
            smooth_method=apply_double_median_filter,
            window_size=21
        )

        true_intervals.append(a_intervals)
        pred_intervals.append(p_intervals)
        trues.append(true_weight_change)
        preds.append(pred_weight_change)
        error_rate = abs(pred_weight_change -
                         true_weight_change) / true_weight_change
        error_rates.append(error_rate)
        # print(f'{user_id} {a_intervals} {p_intervals}\n{true_weight_change:.3f} {pred_weight_change:.3f} \n{error_rate:.3f}')
    except:
        print(f'fail {user_id}')
        continue

  0%|          | 0/59 [00:00<?, ?it/s]

fail 1898


In [7]:
np.nanmedian(error_rates)

0.167913354531201

In [11]:
res = pd.DataFrame({
    'pred_intervals': pred_intervals,
    'true_intervals': true_intervals,
    'pred_weight_change': preds,
    'true_weight_change': trues,
    'error_rate': error_rates
}, index = defecate_ids[(defecate_ids > 1000) & (defecate_ids != 1898)])

res.to_csv('../data/processed/0320-defecate_overall_evaluation_result.csv')

In [24]:
mean_weight_change = np.mean(res['true_weight_change'])
median_weight_change = np.median(res['true_weight_change'])
rmse = np.sqrt(np.mean( (res['pred_weight_change'] - res['true_weight_change']) ** 2 ))
rmse, mean_weight_change, median_weight_change, np.median( np.abs(res['pred_weight_change'] - res['true_weight_change']) )

(0.0623416768649147,
 0.12872988505747282,
 0.0845416666666523,
 0.0189583333333303)

In [17]:
res.loc[1926]

pred_intervals                          [[32, 41]]
true_intervals        [[39.0, 41.0], [33.0, 37.0]]
pred_weight_change                           0.251
true_weight_change                       0.0923333
error_rate                                 1.71841
Name: 1926, dtype: object

In [31]:
res_last_d = pd.read_csv('../data/processed/defecate_overall_evaluation_result_0218.csv', index_col=0)

np.nanmedian(np.abs(res_last_d['Annotated Weight Change'] - res_last_d['Predicted Weight Change']))

0.03108333333333846

In [30]:
res_last_u = pd.read_csv('../data/processed/urinate_overall_evaluation_result_0218.csv', index_col=0)

np.nanmedian(np.abs(res_last_u['Annotated Weight Change'] - res_last_u['Predicted Weight Change']))

0.014666666666677514